<a href="https://colab.research.google.com/github/FarshidRA/farbot/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# بارگذاری مدل و توکنایزر GPT-2
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# تنظیم pad_token
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# بارگذاری داده‌های آموزشی
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# توکن کردن داده‌ها
def tokenize_function(examples):
    encodings = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    return encodings

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# تنظیمات آموزش
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",  # محل ذخیره مدل آموزش‌دیده
    overwrite_output_dir=True,
    num_train_epochs=3,  # تعداد دوره‌های آموزش
    per_device_train_batch_size=2,  # اندازه batch
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",  # محل ذخیره لاگ‌ها
    logging_steps=500,
    report_to="none",  # جلوگیری از ارسال لاگ به WandB
)

# collator برای مدیریت padding
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# ایجاد Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

# شروع آموزش
trainer.train()
